<a href="https://colab.research.google.com/github/yu072333/114-/blob/main/%E7%A8%8B%E8%A8%ADHW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ==============================================================================
# 0. 環境準備與配置
# (請確保您在 Colab 已經安裝了所需的函式庫，例如執行：!pip install gradio gspread pandas plotly google-genai)
# ==============================================================================

# --- 標準函式庫導入 ---
import os
import time
import datetime
import pandas as pd
import plotly.express as px

# --- Colab 專用與 API 函式庫導入 ---
import gradio as gr
import gspread
from google.colab import auth, userdata
from google.auth import default
from google import genai
from google.genai import types

# --- 1. Colab 授權 (必須執行) ---
# 運行後，請按照指示登入並授權 Google 帳號
auth.authenticate_user()

# --- 2. Google Sheets 客戶端初始化 ---
creds, _ = default()
gc = gspread.authorize(creds)


In [2]:
# 推薦在 Colab 筆記本中執行以下命令：
# !pip install gradio gspread pandas plotly google-genai

import os
import gradio as gr
import pandas as pd
import time
import datetime
import plotly.express as px


# 引入 API 庫
import gspread
from google import genai
from google.genai import types
from google.colab import userdata


# =======================================================================

In [3]:

# I. 基礎配置 (請替換成您的設定)
# ==============================================================================
# Google Sheet 配置
# **請將您的完整 Google Sheet 連結貼在這裡！**
SPREADSHEET_URL = 'https://docs.google.com/spreadsheets/d/1Dk1HCgX0yPg3gHSbxZL8bNssT5PaR5NvIRW3K7fd9M4/edit?gid=1885419871#gid=1885419871'
WKS_NAME_TASKS = 'Tasks'
WKS_NAME_POMO_LOG = 'Pomodoro_Log'
WKS_NAME_AI_LOG = 'AI_Analysis_Log'


# Gemini API 配置
GEMINI_API_KEY = userdata.get('gemini')
# 使用獲取的金鑰配置 genai
#genai.configure(api_key=api_key)

#model = genai.GenerativeModel('gemini-2.5-pro')

# 全域番茄鐘控制變數
STOP_TIMER = False # <-- 【加在這裡】
CURRENT_TASK_ID = None
CURRENT_START_DT = None



# =======================================================================


In [4]:
# ==============================================================================
# II. GSheetManager 類 (Google Sheets 互動模組) -
# ==============================================================================

class GSheetManager:
    """管理所有 Google Sheets 的資料讀取與寫入操作"""
    def __init__(self):
        # 1. 宣告要使用全域變數
        global gc
        global SPREADSHEET_URL
        global WKS_NAME_TASKS, WKS_NAME_POMO_LOG, WKS_NAME_AI_LOG # 確保所有配置的全局變量都已宣告

        # 2. 檢查 gc 是否已經初始化（在 Section 0）
        if 'gc' not in globals() or gc is None:
            print("錯誤: gspread 客戶端 'gc' 尚未初始化。請檢查 Section 0。")
            self.sh = None
            self.wks_tasks = None
            self.wks_pomo_log = None
            self.wks_ai_log = None
            return

        # 3. 使用全域的 gc 客戶端來打開 Google Sheet
        try:
            # 使用 URL 打開表格
            self.sh = gc.open_by_url(SPREADSHEET_URL)
            print("Google Sheets 連線成功。")

            # 初始化工作表
            self.wks_tasks = self._get_or_create_worksheet(WKS_NAME_TASKS, ['ID', '任務名稱', '預計時間 (Min)', '難易度', '截止日期', '完成狀態', '總實際工時 (Min)', '備註'])
            self.wks_pomo = self._get_or_create_worksheet(WKS_NAME_POMO_LOG, ['Log_ID', '任務 ID', '任務名稱', '開始時間', '結束時間', '工作時長 (Min)', '休息時長 (Min)'])
            self.wks_ai_log = self._get_or_create_worksheet(WKS_NAME_AI_LOG, ['Timestamp', 'Analysis_Type', 'AI_Prompt', 'AI_Suggestion'])

        except gspread.WorksheetNotFound as e:
            # 處理工作表名稱錯誤
            print(f"錯誤: Google Sheet 工作表名稱錯誤。請檢查配置。錯誤: {e}")
            self.sh = None
        except Exception as e:
            # 這是您當前遇到的錯誤，表示在嘗試開啟時仍有問題
            print(f"Google Sheets 連線失敗: 請檢查 URL 或工作表權限。錯誤: {e}")
            self.sh = None

    def _get_or_create_worksheet(self, name, headers):
        """檢查工作表是否存在，不存在則創建並設定標題"""
        if self.sh is None: return None
        try:
            wks = self.sh.worksheet(name)
        except gspread.WorksheetNotFound:
            print(f"工作表 '{name}' 不存在，正在創建...")
            wks = self.sh.add_worksheet(title=name, rows=100, cols=20)

        if not wks.row_values(1):
             wks.append_row(headers)
        return wks


    def _get_all_records(self, wks):
        """讀取特定工作表資料並轉換為 Pandas DataFrame"""
        if not wks: return pd.DataFrame()
        # 讀取所有資料，使用 get_all_records 讓第一行作為標題
        data = wks.get_all_records()
        return pd.DataFrame(data)


    def get_tasks_df(self):
        return self._get_all_records(self.wks_tasks)

    def get_pomo_log_df(self):
        return self._get_all_records(self.wks_pomo)


    def add_task(self, task_data: dict):
        """新增任務到 Tasks 表"""
        if not self.wks_tasks: return False
        self.wks_tasks.append_row(list(task_data.values()))
        return True

    def log_pomodoro(self, log_data: dict):
        """記錄單次番茄鐘到 Pomodoro_Log 表"""
        if not self.wks_pomo: return False
        # 這裡不需要 Log_ID，因為 log_data 已經有 Log_ID (在 pomodoro_timer_logic 中計算)
        self.wks_pomo.append_row(list(log_data.values()))
        return True


    def update_task_actual_time(self, task_id: int, time_spent_min: int):
        """更新 Tasks 表格中某一任務的總實際工時"""
        if not self.wks_tasks: return "更新失敗：Sheets 未連線。"
        all_data = self.wks_tasks.get_all_values()

        try:
            # 1. 查找目標行號
            target_row_index = [i for i, row in enumerate(all_data) if row and str(row[0]) == str(task_id)][0]
            gsheet_row_num = target_row_index + 1

            # 2. 獲取 '總實際工時 (Min)' 欄位的列索引
            header = all_data[0]
            time_col_index = header.index('總實際工時 (Min)')
            gsheet_col_num = time_col_index + 1

            # 3. 讀取並更新時間
            current_time_str = all_data[target_row_index][time_col_index].strip()
            current_time = float(current_time_str) if current_time_str else 0.0
            new_time = current_time + time_spent_min

            # 4. 寫回新值
            self.wks_tasks.update_cell(gsheet_row_num, gsheet_col_num, new_time)
            return f"任務 ID {task_id} 總工時已更新為 {new_time} 分鐘。"

        except Exception as e:
            return f"更新工時失敗: {e}"

    def delete_task_by_id(self, task_id: int):
        """透過 ID 刪除 Tasks 表中的特定任務"""
        if not self.wks_tasks: return "刪除失敗：Sheets 未連線。"
        all_data = self.wks_tasks.get_all_values()
        try:
            target_row_index = [i for i, row in enumerate(all_data) if row and str(row[0]) == str(task_id)][0]
            gsheet_row_num = target_row_index + 1
            self.wks_tasks.delete_rows(gsheet_row_num)
            return f"成功刪除 ID: {task_id} 的任務。"
        except IndexError:
            return f"錯誤：找不到 ID: {task_id} 的任務。"
        except Exception as e:
            return f"刪除失敗: {e}"


    # --- 資料匯出/匯入 ---
    def export_data(self, format_type: str):
        df = self.get_tasks_df()
        if df.empty: return "無資料可匯出。", None

        file_name = f'tasks_export.{format_type.lower()}'
        if format_type.lower() == 'csv':
            df.to_csv(file_name, index=False)
        elif format_type.lower() == 'json':
            df.to_json(file_name, orient='records', indent=4)
        else: return "不支援的格式。", None

        return f"資料已成功匯出到 {file_name}。", file_name

    def import_data(self, file_path: str):
        try:
            if file_path.lower().endswith('.csv'):
                df = pd.read_csv(file_path)
            elif file_path.lower().endswith('.json'):
                df = pd.read_json(file_path, orient='records')
            else: return "不支援的匯入檔案格式。"


            self.wks_tasks.clear()
            self.wks_tasks.append_row(df.columns.tolist())
            self.wks_tasks.append_rows(df.values.tolist())
            return f"成功從 {file_path} 匯入 {len(df)} 筆資料到 Tasks。"


        except Exception as e:
            return f"匯入失敗: {e}"


    # --- AI Log 寫入 ---
    def log_ai_analysis(self, analysis_type, prompt, suggestion):
        if not self.wks_ai_log: return False
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        self.wks_ai_log.append_row([timestamp, analysis_type, prompt, suggestion])
        return True


In [5]:

# III. GeminiAssistant 類 (AI 助理模組)
# ==============================================================================


class GeminiAssistant:
    """整合 Gemini API 進行時間管理分析與建議"""
    def __init__(self):
        self.client = None
        if not GEMINI_API_KEY:
            print("警告：GEMINI_API_KEY 未找到，AI 助理功能將無法使用。")
            return

        try:
            self.client = genai.Client(api_key=GEMINI_API_KEY)
            self.model = 'gemini-2.5-flash'
            print("Gemini API 連線成功。")
        except Exception as e:
            print(f"Gemini API 連線失敗: {e}")
            self.client = None


    def analyze_time_data(self, tasks_df: pd.DataFrame, pomo_log_df: pd.DataFrame, user_prompt: str):
        """分析任務數據和番茄鐘紀錄，提供建議。"""
        if self.client is None:
            return "AI 助理未初始化，請檢查 API Key。"


        tasks_context = tasks_df.to_markdown(index=False)
        pomo_context = pomo_log_df.to_markdown(index=False)

        system_instruction = (
            "你是一位專業的時間管理與效率分析師。請根據提供的任務數據 (Tasks) 和詳細的工作紀錄 (Pomodoro_Log)，"
            "提供具體的效率提升建議和時間分配策略。請以條列式清晰回答，並使用中文回應。"
        )


        full_prompt = (
            f"以下是我的任務列表 (Tasks):\n{tasks_context}\n\n"
            f"以下是我的詳細工作紀錄 (Pomodoro Log):\n{pomo_context}\n\n"
            f"--- 請分析上述數據並回答以下問題與建議：---\n"
            f"1. 總結我的時間分配特點，並指出預計時間與實際工時的差異。\n"
            f"2. 針對我效率最低或花費時間最長的任務，提供具體的效率提升建議。\n"
            f"3. {user_prompt} (使用者提出的額外問題)\n\n"
        )


        try:
            response = self.client.models.generate_content(
                model=self.model,
                contents=full_prompt,
                config=types.GenerateContentConfig(
                    system_instruction=system_instruction
                )
            )
            return response.text
        except Exception as e:
            return f"Gemini API 呼叫失敗: {e}"


#


In [6]:
# ==============================================================================
# IV. 應用程式功能函數 (整合 Gradio 與 Manager)
# ==============================================================================

# 初始化管理器和助理 (假設 GSheetManager 和 GeminiAssistant 類已定義)
gs_manager = GSheetManager()
ai_assistant = GeminiAssistant()

# --- 核心任務管理函數 ---

def refresh_tasks_table():
    """刷新任務列表，確保即使沒資料也有正確標題"""
    df = gs_manager.get_tasks_df()
    if df.empty:
        # 使用空的 Dataframe 保持 Gradio 表格結構
        return pd.DataFrame(columns=['ID', '任務名稱', '預計時間 (Min)', '難易度', '截止日期', '完成狀態', '總實際工時 (Min)', '備註'])
    return df


def add_task_ui(name, time_cost, difficulty, deadline):
    """新增任務並計算 ID"""
    if not deadline:
        return "錯誤：截止日期不能為空。", refresh_tasks_table()

    # 處理 Gradio 日期物件
    deadline_str = ""
    try:
        if isinstance(deadline, datetime.datetime):
            deadline_str = deadline.strftime("%Y-%m-%d")
        else:
            deadline_str = str(deadline).split(' ')[0]
    except Exception as e:
        return f"日期格式錯誤: {e}", refresh_tasks_table()

    current_df = gs_manager.get_tasks_df()
    max_id = current_df['ID'].max() if 'ID' in current_df.columns and not current_df.empty else 0
    new_id = int(max_id) + 1

    new_task = {
        'ID': new_id,
        '任務名稱': name,
        '預計時間 (Min)': int(time_cost),
        '難易度': difficulty,
        '截止日期': deadline_str,
        '完成狀態': '待辦',
        '總實際工時 (Min)': 0,
        '備註': ''
    }

    if gs_manager.add_task(new_task):
        return "任務新增成功！", refresh_tasks_table()
    else:
        return "任務新增失敗：Google Sheet 連線或寫入錯誤。", refresh_tasks_table()


def delete_task_ui(task_id_str):
    """刪除任務"""
    try:
        task_id = int(task_id_str)
        result_msg = gs_manager.delete_task_by_id(task_id)
        return result_msg, refresh_tasks_table()
    except ValueError:
        return "錯誤：ID 必須是整數。", refresh_tasks_table()


def update_task_status(updated_data):
    """將更新後的 DataFrame 寫回 Google Sheet，允許編輯狀態"""
    if not gs_manager.sh:
        return "錯誤：Sheets 連線失敗，無法更新。", refresh_tasks_table()

    updated_df = pd.DataFrame(updated_data, columns=['ID', '任務名稱', '預計時間 (Min)', '難易度', '截止日期', '完成狀態', '總實際工時 (Min)', '備註'])

    try:
        # 1. 清空原始工作表 (除標題行)
        gs_manager.wks_tasks.delete_rows(2, gs_manager.wks_tasks.row_count)

        # 2. 寫入新的資料
        gs_manager.wks_tasks.append_rows(updated_df.values.tolist(), value_input_option='USER_ENTERED')
        return "任務列表已更新。", updated_df
    except Exception as e:
        return f"更新失敗：{e}", refresh_tasks_table()


def query_tasks(status, start_date, end_date):
    """查詢任務"""
    df = gs_manager.get_tasks_df()
    # (查詢邏輯不變，保持您提供的版本)
    if df.empty: return "無資料可查詢。", pd.DataFrame()

    filtered_df = df.copy()

    if status and status != "全部":
        filtered_df = filtered_df[filtered_df['完成狀態'] == status]

    if start_date and end_date:
        filtered_df['截止日期'] = pd.to_datetime(filtered_df['截止日期'], errors='coerce')
        start_dt = pd.to_datetime(start_date)
        end_dt = pd.to_datetime(end_date)
        filtered_df = filtered_df[
            (filtered_df['截止日期'] >= start_dt) &
            (filtered_df['截止日期'] <= end_dt)
        ]
        filtered_df['截止日期'] = filtered_df['截止日期'].dt.strftime('%Y-%m-%d')

    return f"查詢到 {len(filtered_df)} 筆結果。", filtered_df.drop(columns=['ID'])

# --- 番茄鐘邏輯函數 ---

def log_and_clear_pomodoro_state(break_min=0):
    """
    記錄工時到 Google Sheet 並清除全域狀態。
    當計時器完成或被手動停止時呼叫。
    """
    global CURRENT_TASK_ID, CURRENT_START_DT

    if CURRENT_TASK_ID is None or CURRENT_START_DT is None:
        return "錯誤：無正在進行的任務或開始時間。", False

    end_work_dt = datetime.datetime.now()
    task_id = CURRENT_TASK_ID

    # 計算實際花費時間
    time_spent_timedelta = end_work_dt - CURRENT_START_DT
    time_spent_min = round(time_spent_timedelta.total_seconds() / 60, 2)

    try:
        # 重新獲取任務名稱
        task_name = gs_manager.get_tasks_df()[gs_manager.get_tasks_df()['ID'] == task_id].iloc[0]['任務名稱']
    except IndexError:
        task_name = "未知任務" # 處理任務可能已被刪除的情況

    log_data = {
        'Log_ID': int(len(gs_manager.get_pomo_log_df())) + 1,
        '任務 ID': int(task_id), # 確保為標準 Python int
        '任務名稱': task_name,
        '開始時間': CURRENT_START_DT.strftime("%Y-%m-%d %H:%M:%S"),
        '結束時間': end_work_dt.strftime("%Y-%m-%d %H:%M:%S"),
        '工作時長 (Min)': float(time_spent_min), # 確保為標準 Python float
        '休息時長 (Min)': float(break_min)
    }

    gs_manager.log_pomodoro(log_data)
    gs_manager.update_task_actual_time(task_id, time_spent_min)

    # 清除全域狀態
    CURRENT_TASK_ID = None
    CURRENT_START_DT = None

    return f"已記錄 {time_spent_min} 分鐘工時。", True


def stop_pomodoro_ui():
    """
    停止計時器按鈕的處理函數。
    它會呼叫 log_and_clear_pomodoro_state 記錄實際已花費的時間。
    """
    # 此處不需要 global STOP_TIMER = True，因為我們依賴 log_and_clear_pomodoro_state 進行記錄

    msg, success = log_and_clear_pomodoro_state(break_min=0)

    if success:
        return f"**[已停止]** {msg}", "00:00"
    else:
        return f"**[停止錯誤]** {msg}", "00:00"


def pomodoro_timer_logic(selected_task_name):
    """番茄鐘主要計時邏輯 (Generator)"""
    global CURRENT_TASK_ID, CURRENT_START_DT

    if not selected_task_name or selected_task_name == "無任務":
        yield "請選擇一個任務開始計時！", "00:00"
        return

    # 獲取任務 ID (此處需要重新獲取，因為我們沒有將 task_info 儲存為全局)
    TASK_DF = gs_manager.get_tasks_df()
    try:
        task_info = TASK_DF[TASK_DF['任務名稱'] == selected_task_name].iloc[0]
        task_id = int(task_info['ID'])
    except:
        yield f"錯誤：找不到任務 '{selected_task_name}' 的 ID。", "00:00"
        return

    # 標準時間設定
    WORK_MIN = 25
    BREAK_MIN = 5
    # TEST_MODE = False # 假設您已經關閉

    # ------------------- 狀態追蹤 -------------------
    # 在計時開始時初始化全局狀態
    CURRENT_TASK_ID = task_id
    CURRENT_START_DT = datetime.datetime.now()
    # ------------------------------------------------

    # 1. 工作階段
    yield f"**[工作中]** 任務: {selected_task_name}. 倒數 {WORK_MIN:02d}:00", f"{WORK_MIN:02d}:00"

    work_seconds = int(WORK_MIN * 60)

    for i in range(work_seconds):
        # 由於我們使用 cancels=[start_event]，如果 stop 被按下，這個生成器會在這裡被中斷。
        remaining = work_seconds - i
        minutes = remaining // 60
        seconds = remaining % 60

        yield gr.update(value=f"**[工作中]** 任務: {selected_task_name}. 剩餘 {minutes:02d}:{seconds:02d}"), gr.update(value=f"{minutes:02d}:{seconds:02d}")
        time.sleep(1)

    # 【工作完成：自動記錄工時】
    log_msg, success = log_and_clear_pomodoro_state(break_min=BREAK_MIN)
    if not success:
         yield f"番茄鐘錯誤：{log_msg}", "00:00"
         return

    # 2. 休息階段 (只有工作完成後才進入)
    yield f"**[休息中]** 休息 {int(BREAK_MIN):02d}:00", f"{int(BREAK_MIN):02d}:00"

    for i in range(int(BREAK_MIN * 60)):
        remaining = int(BREAK_MIN * 60) - i
        minutes = remaining // 60
        seconds = remaining % 60
        yield gr.update(value=f"**[休息中]** 剩餘 {minutes:02d}:{seconds:02d}"), gr.update(value=f"{minutes:02d}:{seconds:02d}")
        time.sleep(1)

    yield "番茄鐘循環結束，工時已記錄！請刷新任務列表查看。", "00:00"

# --- 數據分析與 AI 助理函數 (不變) ---

def visualize_data():
    """生成任務時間花費與完成率的 Plotly 圖表"""
    # ... (此函數邏輯保持不變)
    df_tasks = gs_manager.get_tasks_df()

    if df_tasks.empty:
        return gr.Plot(visible=False), gr.Plot(visible=False)


    df_tasks['總實際工時 (Min)'] = pd.to_numeric(df_tasks['總實際工時 (Min)'], errors='coerce').fillna(0)

    # 1. 任務花費時間長條圖
    fig_time = px.bar(
        df_tasks.sort_values(by='總實際工時 (Min)', ascending=False).head(10),
        x='任務名稱',
        y='總實際工時 (Min)',
        title='任務實際花費時間 (前10)'
    )

    # 2. 任務完成率圓餅圖
    completion_counts = df_tasks['完成狀態'].value_counts().reset_index()
    completion_counts.columns = ['Status', 'Count']
    fig_completion = px.pie(
        completion_counts,
        values='Count',
        names='Status',
        title='任務完成狀態分佈'
    )

    return gr.Plot(fig_time, visible=True), gr.Plot(fig_completion, visible=True)


def run_ai_analysis_ui(user_prompt):
    """呼叫 Gemini 助理進行分析並記錄"""
    # ... (此函數邏輯保持不變)
    df_tasks = gs_manager.get_tasks_df()
    df_pomo = gs_manager.get_pomo_log_df()

    if df_tasks.empty:
        return "Google Sheets 中沒有任務數據，無法進行分析。", gr.Markdown("")


    ai_response = ai_assistant.analyze_time_data(df_tasks, df_pomo, user_prompt)


    # 記錄到 Sheets
    gs_manager.log_ai_analysis("Time_Management_Analysis", user_prompt, ai_response)


    return "AI 分析已完成並記錄到 Google Sheet。", gr.Markdown(f"### AI 分析建議:\n\n{ai_response}")

# --- 數據匯入/匯出函數 ---

def export_tasks_data(file_format):
    """將任務數據匯出為 CSV 或 JSON 檔案"""
    df = gs_manager.get_tasks_df()

    if df.empty:
        # Gradio 的 gr.File 輸出需要 None 來表示沒有檔案
        return None

    # 建立臨時檔案
    filename = f"task_records_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.{file_format.lower()}"
    # 使用 /tmp 目錄 (在 Colab/雲端環境中這是寫入臨時檔案的標準位置)
    filepath = os.path.join("/tmp", filename)

    try:
        if file_format.upper() == 'CSV':
            df.to_csv(filepath, index=False, encoding='utf-8')
        elif file_format.upper() == 'JSON':
            # orient='records' 是常見的 JSON 格式
            df.to_json(filepath, orient='records', force_ascii=False)
        else:
            return None

        # Gradio File 組件輸出需要檔案路徑
        return filepath
    except Exception as e:
        print(f"匯出失敗: {e}")
        return None

def import_tasks_data(file_obj):
    """從 CSV/JSON 檔案匯入數據並寫入 Google Sheet"""
    if file_obj is None:
        return "錯誤：請先上傳檔案！"

    filepath = file_obj.name
    filename = os.path.basename(filepath)

    try:
        if filename.lower().endswith('.csv'):
            df_new = pd.read_csv(filepath)
        elif filename.lower().endswith('.json'):
            df_new = pd.read_json(filepath)
        else:
            return "錯誤：只支持 CSV 和 JSON 檔案！"

        # 檢查欄位是否匹配
        required_cols = ['ID', '任務名稱', '預計時間 (Min)', '難易度', '截止日期', '完成狀態', '總實際工時 (Min)', '備註']
        if not all(col in df_new.columns for col in required_cols):
            return f"錯誤：匯入檔案缺少必需的欄位。需有：{', '.join(required_cols)}"

        # 寫入 Google Sheet (覆蓋現有數據，所以需要清空再寫入)
        # 由於您可能只有一個 wks_tasks，這裡假設您只需要替換任務表
        gs_manager.wks_tasks.delete_rows(2, gs_manager.wks_tasks.row_count)
        # 確保所有數據類型正確，特別是數字欄位
        df_new['ID'] = pd.to_numeric(df_new['ID'], errors='coerce').fillna(0).astype(int)

        gs_manager.wks_tasks.append_rows(df_new.values.tolist(), value_input_option='USER_ENTERED')

        return f"✅ 數據匯入成功！{len(df_new)} 條記錄已寫入 Google Sheet。"

    except Exception as e:
        return f"匯入失敗：{e}"


Google Sheets 連線成功。
Gemini API 連線成功。


In [ ]:
# ==============================================================================
# V. Gradio 介面定義與運行
# ==============================================================================

# ----------------- UI 介面元素定義 -----------------

# 任務管理 Tab
tasks_table = gr.Dataframe(
    headers=['ID', '任務名稱', '預計時間 (Min)', '難易度', '截止日期', '完成狀態', '總實際工時 (Min)', '備註'],
    label="當前任務列表",
    interactive=True,
    row_count=5,
    col_count=(8, 'fixed')
)
add_msg = gr.Markdown("填寫任務資訊並點擊新增", elem_id="add_task_message")
delete_msg = gr.Markdown("輸入 ID 刪除任務", elem_id="delete_task_message")
query_msg = gr.Markdown("篩選結果", elem_id="query_message")
query_table = gr.Dataframe(label="查詢結果", interactive=False, visible=True)


# 番茄鐘 Tab
task_names = [] # 這裡將在介面啟動前或刷新時更新
task_selector = gr.Dropdown(label="選擇任務", choices=task_names, value="無任務")
timer_display = gr.Textbox("00:00", label="計時器", show_label=True, interactive=False, elem_id="timer-display")
status_message = gr.Markdown("**準備就緒**", elem_id="pomo_status_message")


# 數據分析 Tab
chart_time = gr.Plot(label="任務時間花費分佈")
chart_completion = gr.Plot(label="任務完成狀態分佈")
analysis_prompt = gr.Textbox("請特別分析我上週最常拖延的任務，並提供改進方法。", label="給 AI 的提示")
analysis_result_markdown = gr.Markdown("### AI 分析建議:\n\n請輸入提示並點擊分析按鈕。", label="AI 分析結果")
analysis_status = gr.Textbox(label="AI 助理狀態", interactive=False)

#CSV/JSON
download_file = gr.File(label="下載您的數據紀錄", file_count='single', visible=True, interactive=False)
import_file = gr.File(label="選擇 CSV/JSON 檔案匯入", file_types=['.csv', '.json'], file_count='single')
import_status = gr.Markdown("等待匯入...")


# ----------------- Gradio 應用程式定義 -----------------

with gr.Blocks(title="番茄鐘與時間管理助理") as demo:
    gr.HTML("<h1 style='text-align: center; color: #4CAF50;'>🍅 時間管理助理 (Gemini & GSheets)</h1>")

    with gr.Tab("任務管理"):
        with gr.Row():
            with gr.Column(scale=2):
                gr.Markdown("## 1. 新增任務")
                task_name_input = gr.Textbox(label="任務名稱")
                with gr.Row():
                    time_cost_input = gr.Number(label="預計時間 (Min)", value=30)
                    difficulty_input = gr.Dropdown(["低", "中", "高"], label="難易度", value="中")
                deadline_input = gr.DateTime(label="截止日期", type="datetime")
                add_btn = gr.Button("新增任務")
                add_msg.render()

            with gr.Column(scale=1):
                gr.Markdown("## 2. 刪除任務")
                delete_id_input = gr.Textbox(label="任務 ID")
                delete_btn = gr.Button("刪除任務")
                delete_msg.render()

        gr.Markdown("## 3. 任務列表")
        refresh_btn = gr.Button("刷新任務列表")
        tasks_table.render()

        with gr.Accordion("進階查詢", open=False):
            with gr.Row():
                status_query = gr.Dropdown(["全部", "待辦", "進行中", "已完成"], label="狀態")
                start_date_query = gr.Textbox(label="開始日期 (YYYY-MM-DD)")
                end_date_query = gr.Textbox(label="結束日期 (YYYY-MM-DD)")
            query_btn = gr.Button("查詢任務")
            query_msg.render()
            query_table.render()


    with gr.Tab("番茄鐘"):
        gr.Markdown("## 番茄鐘計時器")
        with gr.Row():
            task_selector.render()
            start_pomo_btn = gr.Button("開始番茄鐘 ")
            stop_pomo_btn = gr.Button("停止計時並記錄", variant="stop")

        status_message.render()
        timer_display.render()

        gr.Markdown("---")
        gr.Markdown("工作結束後，請點擊 **刷新任務列表** 查看工時更新！")


    with gr.Tab("數據分析 & AI 助理"):
        gr.Markdown("## 數據視覺化")
        with gr.Row():
            chart_time.render()
            chart_completion.render()

        visualize_btn = gr.Button("生成圖表")

        # 【新增匯入/匯出 UI】
    gr.Markdown("---")
    gr.Markdown("## 數據匯入與匯出 (CSV/JSON)")
    with gr.Row():
        export_btn_csv = gr.Button("匯出所有任務 (CSV)")
        export_btn_json = gr.Button("匯出所有任務 (JSON)")

    download_file.render()

    gr.Markdown("#### ⚠ 匯入數據 (將覆蓋現有數據，請謹慎操作)")
    import_file.render()
    import_btn = gr.Button("執行數據匯入", variant='stop')
    import_status.render()

    gr.Markdown("---")
    gr.Markdown("## AI 時間管理分析")
    analysis_prompt.render()
    run_ai_btn = gr.Button("運行 AI 分析並記錄")
    analysis_status.render()
    analysis_result_markdown.render()



    # ----------------- 介面互動邏輯 (Events) -----------------

    # 載入時和刷新時的邏輯
    def initial_load():
        df = refresh_tasks_table()
        task_names = df['任務名稱'].tolist() if not df.empty else ["無任務"]
        return df, gr.update(choices=task_names, value=task_names[0] if task_names else "無任務")

    demo.load(initial_load, None, [tasks_table, task_selector])
    refresh_btn.click(initial_load, None, [tasks_table, task_selector])

    # 任務狀態更新邏輯 (當用戶編輯表格時觸發)
    tasks_table.change(
        update_task_status,
        [tasks_table],
        [delete_msg, tasks_table] # 用 delete_msg 顯示更新訊息
    )

    # 任務操作邏輯
    add_btn.click(
        add_task_ui,
        [task_name_input, time_cost_input, difficulty_input, deadline_input],
        [add_msg, tasks_table]
    ).then(initial_load, None, [tasks_table, task_selector]) # 新增後刷新下拉選單

    delete_btn.click(
        delete_task_ui,
        [delete_id_input],
        [delete_msg, tasks_table]
    ).then(initial_load, None, [tasks_table, task_selector]) # 刪除後刷新下拉選單

    # 查詢邏輯
    query_btn.click(
        query_tasks,
        [status_query, start_date_query, end_date_query],
        [query_msg, query_table]
    )

    # 番茄鐘邏輯
    start_event = start_pomo_btn.click(
        pomodoro_timer_logic,
        [task_selector],
        [status_message, timer_display]
    )
    # 【新增停止按鈕邏輯】
    stop_pomo_btn.click(
        stop_pomodoro_ui,
        None,
        [status_message, timer_display],
        # 這個關鍵字參數很重要，它會阻止 start_pomo_btn 的 generator 繼續運行
        cancels=[start_event]
    )
# ----------------- 數據匯入/匯出邏輯 (新增) -----------------
    export_btn_csv.click( # <--- 修正這行
        export_tasks_data,
        gr.State("CSV"),
        [download_file]
    )
    export_btn_json.click( # <--- 修正這行
        export_tasks_data,
        gr.State("JSON"),
        [download_file]
    )

    import_btn.click( # <--- 修正這行
        import_tasks_data,
        [import_file],
        [import_status]
    ).then(initial_load, None, [tasks_table, task_selector]) # 匯入後刷新任務列表

    # 數據分析邏輯
    visualize_btn.click(visualize_data, None, [chart_time, chart_completion])

    # AI 助理邏輯
    run_ai_btn.click(
        run_ai_analysis_ui,
        [analysis_prompt],
        [analysis_status, analysis_result_markdown]
    )
    # 運行 Gradio 介面
    if __name__ == "__main__":

      try:
        # 建議在 Gradio 中新增 share=True，方便在 Colab 或遠端環境中使用
        demo.launch(debug=True, share=True)
      except Exception as e:
        print(f"Gradio 啟動失敗：{e}")
        print("請確認您已安裝所有函式庫 (`!pip install gradio gspread pandas plotly google-genai`)")


/usr/local/lib/python3.12/dist-packages/gradio/components/dropdown.py:230: UserWarning: The value passed into gr.Dropdown() is not in the list of choices. Please update the list of choices to include: 無任務 or set allow_custom_value=True.
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://3170a04b2113020816.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
